In [ ]:
# !wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
# !unzip ml-latest-small.zip

--2024-04-16 19:08:00--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K   988KB/s    in 1.0s    

2024-04-16 19:08:03 (988 KB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [1]:
import numpy as np

users = []
movies = []
ratings = []

with open("ml-latest-small/ratings.csv", "r") as f:
    print(f.readline())

    for line in f:
        uid, mid, rating, _ = line.split(",")
        users.append(int(uid))
        movies.append(int(mid))
        ratings.append(float(rating))

users = np.array(users)
movies = np.array(movies)
ratings = np.array(ratings)

userId,movieId,rating,timestamp



In [2]:
rank = 10
lr = 0.1
lmd = 0.001

In [3]:
alpha = ratings.mean()
user_bias = np.zeros(users.max() + 1)
movie_bias = np.zeros(movies.max() + 1)
user_reps = np.random.randn(users.max() + 1, rank)
movie_reps = np.random.randn(movies.max() + 1, rank)

In [4]:
for epoch in range(10):
    
    h = alpha + user_bias[users] + movie_bias[movies] + (user_reps[users] * movie_reps[movies]).sum(axis=1)
    rmse = ((h - ratings) ** 2).mean() ** 0.5
    print(f"epoch: {epoch}, rmse: {rmse}")

    for user, movie, rating in zip(users, movies, ratings):
        bu = user_bias[user]
        bi = movie_bias[movie]
        ru = user_reps[user]
        ri = movie_reps[movie]

        # 예측값
        est = alpha + bu + bi + np.dot(ru, ri)
        diff = est - rating

        # gradient descent
        grd_user_bias = diff + lmd * bu
        grd_movie_bias = diff + lmd * bi
        grd_user_reps = diff * ri + lmd * ru
        grd_movie_reps = diff * ru + lmd * ri

        # update
        user_bias[user] -= lr * grd_user_bias
        movie_bias[movie] -= lr * grd_movie_bias
        user_reps[user] -= lr * grd_user_reps
        movie_reps[movie] -= lr * grd_movie_reps

epoch: 0, rmse: 3.291827339705382
epoch: 1, rmse: 1.14157228441919
epoch: 2, rmse: 1.0301441900581474
epoch: 3, rmse: 0.9850881013439668
epoch: 4, rmse: 0.9616059402000787
epoch: 5, rmse: 0.9479119124925233
epoch: 6, rmse: 0.9374234102014337
epoch: 7, rmse: 0.9279278454881971
epoch: 8, rmse: 0.9196263595166057
epoch: 9, rmse: 0.9131980798930296


In [1]:
# !pip3 install surprise

Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit-surprise-1.1.3.tar.gz (771 kB)
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-macosx_10_9_universal2.whl size=1607959 sha256=a431db1727abb78a6b55ca6fb2bf682609b379b61fd9626a4bc97c7ddcc4e97f
  Stored in directory: /Users/ull/Library/Caches/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [16]:
from surprise import SVD, Reader, Dataset
from surprise.model_selection import train_test_split

reader = Reader(line_format="user item rating timestamp", sep=",", skip_lines=1)
data = Dataset.load_from_file("ml-latest-small/ratings.csv", reader=reader)

trainset, testset = train_test_split(data, test_size=0.1)

In [17]:
model = SVD(n_factors=10)

model.fit(trainset)

model.predict('31', '1523')

Prediction(uid='31', iid='1523', r_ui=None, est=3.854883330512808, details={'was_impossible': False})

In [18]:
from surprise import accuracy

train_to_test = trainset.build_testset()
pred_train = model.test(train_to_test)
train_rmse = accuracy.rmse(pred_train, verbose=False)

pred_test = model.test(testset)
test_rmse = accuracy.rmse(pred_test, verbose=False)
print(f"train_rmse: {train_rmse}, test_rmse: {test_rmse}")

train_rmse: 0.7971972564998103, test_rmse: 0.8644636146547608
